# Getting Started - Vanillix Tutorial

In this tutorial, we'll demonstrate how to use the basic functionalities of our **AUTOENCODIX** pipeline.  
We’ll cover the general workkflow, show how to work with custom classes, and highlight the pipeline’s flexibility.  
**AUTOENCODIX** supports far more functionality than shown here, so we’ll also point to advanced tutorials where relevant.  

**IMPORTANT**

> This tutorial serves two purposes: (a) introducing the framework in general, and (b) presenting the Vanillix pipeline.  
> For other architectures (e.g., Varix or Ontix), we’ll skip the general concepts and focus only on their specifics.  
> To get a solid overview of the framework, we recommend starting here. We’ll include this note in other tutorials as well, so you can always find the right entry point.

---
## What You'll Learn

You’ll learn how to:

1. **Initialize** the pipeline with a pandas DataFrame. <br><br>
2. Understand the **pipeline steps**. <br><br>
3. Access the pipeline **results** (latent spaces, losses, etc.). <br><br>
4. **Visualize** outputs effectively. <br><br>
5. Apply **custom parameters**. <br><br>
6. **Save, load, and reuse** a trained pipeline. <br><br>

---

Let’s get started! 🚀

## 1) Initialization

To get started, you need three things:  
- A dataset  
- A configuration object  
- The Vanillix pipeline  

---

#### 📊 Dataset
We allow a variety of inputs (see [1]).  
Here, we demonstrate how to use the pipeline with pandas DataFrames.  

We prepared three mock DataFrames to simulate a multi-omics use case:  
- RNA: `raw_rna`  
- Protein: `raw_protein`  
- Clinical metadata: `annotation`  

For simplicity, we assume paired metadata.  
For handling unpaired data, see [1] or [2]. <br><br>

---

#### ⚙️ Configuration Object
We use the pre-set parameters in `VanillixConfig`.  

Additionally, we define a `DataCase`, which tells the pipeline what kind of data to expect.  
In this case: `MULTI_BULK`.  

This can be any tabular data and does not have to be bulk sequencing data (we may rename this soon).  
For more details on `DataCase`, refer to [1]. <br><br>

---

#### 🪈 Vanillix Pipeline
Finally, we import `Vanillix` from our package.  

Each autoencoder architecture comes with a corresponding `pipeline`, which handles the complete analysis process—from preprocessing to evaluation.  
All autoencoders share this unified pipeline interface, which we’ll explore in the next sections. <br><br>

---

[1] Tutorials/DeepDives/InputDataTutorials.ipynb <br>  
[2] Tutorials/PipelineTutorials/XModalix.ipynb <br>


### 1.1) Inspect Data
Before initializing the pipeline, we’ll show what the input data can look like.  
This should help when working with your own data later.


In [1]:
from autoencodix.utils.example_data import raw_protein, raw_rna, annotation

print(f"Type of raw_rna input: {type(raw_rna)}")

Type of raw_rna input: <class 'pandas.core.frame.DataFrame'>


In [2]:
raw_rna.head()

,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,gene_9,...,gene_90,gene_91,gene_92,gene_93,gene_94,gene_95,gene_96,gene_97,gene_98,gene_99
sample_0,-0.693594,-0.473079,0.676121,-0.147388,1.201123,-0.414951,0.753255,-1.108531,2.158595,2.086612,...,0.393002,1.086796,-4.323870,2.633942,2.448690,0.091798,2.143187,3.864588,4.332923,-2.012073
sample_1,-7.717414,4.786169,-5.157288,-2.602279,-0.652874,-0.199347,0.883363,-1.917114,2.130440,2.577293,...,-1.764460,1.574903,-6.280627,0.727504,3.392857,1.263027,0.853917,-3.103998,0.276629,-2.708227
sample_2,3.318892,-2.814952,5.125116,-0.222650,1.384565,-5.355473,-2.930422,-1.684189,4.366761,-0.364140,...,0.055857,-0.903358,1.608255,4.002976,0.399693,-0.791146,-2.655789,-0.346798,0.118127,-0.097970
sample_3,0.305501,-2.003316,-5.919967,3.262441,-3.607759,4.005525,3.349946,0.840550,0.173683,-4.018448,...,0.866646,5.055061,-1.797290,-3.199909,3.422235,2.897288,1.775569,0.937340,-2.042406,2.651122
sample_4,-2.942519,1.831377,-1.003791,3.505788,-0.765147,-2.217828,-2.149387,1.377427,-0.134418,1.960087,...,-4.148493,1.730545,-3.276235,0.263396,1.315254,-1.933574,0.764190,-0.877114,-2.640165,-5.108126


In [3]:
annotation.head()

,condition,batch,quality_score
sample_0,condition_0,batch_3,0.883087
sample_1,condition_0,batch_2,0.884084
sample_2,condition_1,batch_2,0.971995
sample_3,condition_1,batch_1,0.718440
sample_4,condition_0,batch_3,0.854322


### 1.2) Initialize Pipeline
In this step, we import the pipeline, configuration, and data classes, and show how to feed pandas DataFrames into the pipeline.  

We use the `DataPackage` class for this purpose. In our case, we fill the attribute `multi_bulk` with a dictionary of pandas DataFrames.  
*Note:* the term *multi* may be misleading—you can also provide just a single DataFrame.  

Metadata is also passed as a dictionary to the `DataPackage`.  
Here, we have one annotation DataFrame for both data modalities (RNA and protein), so we use the identifier `"paired"` as the key.  
Otherwise, you would use the same keys as for the actual data, e.g.:  
```python
{"rna": rna_annotation, "protein": protein_annotation}


In [4]:
import autoencodix as acx
from autoencodix.data.datapackage import DataPackage
from autoencodix.configs.vanillix_config import VanillixConfig
from autoencodix.configs.default_config import DataCase

# If your data is stored in pandas DataFrames, you can easily pass them to our custom DataPackage.
# For any tabular data that is not single-cell, provide them as a dictionary to the "multi_bulk" attribute of DataPackage.
# Note: "multi" might be misleading—it's valid to provide just one modality (1–n data modalities).
# Here we assume paired metadata. If you have separate metadata for each modality, use the same dict keys as in multi_bulk, e.g.:
# annotation = {"rna": rna_annotation, "protein": protein_annotation}
my_datapackage: DataPackage = DataPackage(
    multi_bulk={"rna": raw_rna, "protein": raw_protein},
    annotation={"paired": annotation},
)

myconfig: VanillixConfig = VanillixConfig(data_case=DataCase.MULTI_BULK, epochs=30)
vanillix = acx.Vanillix(data=my_datapackage, config=myconfig)


in handle_direct_user_data with data: <class 'autoencodix.data.datapackage.DataPackage'>




## 2) Pipeline Steps

Next, we explain the four main steps of our pipeline:

1. Preprocess
2. Fit
3. Predict
4. Visualize

We can run all steps together with the `run` step.

### 2.1) Preprocess

If you're familiar with `AUTOENCODIX 1.0`, this is similar to `make data` in the old framework.  
In case you don't want to have any preprocessing, you can set `skip_preprocessing=True` in the config object.  
ATTENTION: This could lead to `RunTimeErrors` if you violate assumptions like NaN-free data, which we'll handle in preprocessing.  
See Tutorial [1] for details on data requirements and assumptions.
#### 🔗 Data Pairing

If you have multiple data modalities, i.e., mRNA and CNA data, and you work with a pipeline that requires paired data (Vanillix, Varix, Ontix, Disentanglix), we drop all samples that are not present in all data modalities.
If you want to include unpaired data, please refer to [2] and [3].

#### 🧼 Data Cleaning

This step acts as a safety net to avoid runtime errors later in the pipeline. Here, we simply drop columns with NaN values.
Ideally, you’ve already cleaned the data before putting it into our pipeline if you want more sophisticated handling of NaNs.
**Note:** For metadata, we allow NaNs as long as they’re not specified in the config as relevant columns (see [4]).

#### ✂️ Splitting

We designed the preprocessing pipeline to avoid data leakage, so almost all preprocessing transformations are applied after splitting the data.
The only exception is filtering cells and log-transforming SingleCell data (if not already done by you).
The default behavior is to split your input data into three splits: train, validation, and test. You can define the ratio via the Config (see [4]) or, for some pipelines, provide a custom split (see [1]).
Additionally, you always pass unseen data to the `predict` step of the pipeline.

#### 🗂️ Filtering

We filter the columns of the train set and apply the learned filtering to the validation and test sets. The filtering method and the number of features to keep can be defined in the config.

The filtering method can be set per data modality, but we recommend keeping it consistent.
If you work with multiple data modalities and provide a number for `k_filter` (e.g., 1000), this will be the total number of features across all data modalities. For example, you may have 500 features for data modality A and 500 features for data modality B.

We provide the following options for filtering:

- `VAR` – Select features with the highest variance
- `MAD` – Select features with the highest median absolute deviation
- `CORR` – Select features based on correlation
- `VARCORR` – Combination of variance and correlation
- `NOFILT` – No filtering applied
- `NONZEROVAR` – Select features with non-zero variance

#### 📏 Scaling
After the filtering step, we scale the data. We fit the scaler on the train set and apply the learned transformations to the validation and test sets.  

Different scaling methods can be applied per data modality, but we strongly recommend setting this globally via the `scaling` parameter. The default is `StandardScaler`.  

We provide the following options for scaling:  
- `STANDARD` – Standard scaling (mean = 0, variance = 1)  
- `MINMAX` – Min-max scaling to range [0, 1]  
- `ROBUST` – Robust scaling using median and interquartile range  
- `MAXABS` – Scaling by the maximum absolute value  
- `NONE` – No scaling applied

#### 🐍 Transforming to PyTorch Dataset 
In the last step, we transform the data to a custom PyTorch Dataset class. 
For the Vanillix pipeline we concat the different data modalites and store them in the data attribute of Dataset class.  
Now the data is ready to serve as input for our PyTorch models.

---

[1] Tutorials/DeepDives/InputDataTutorials.ipynb <br>
[2] Tutorials/PipelineTutorials/XModalix.ipynb <br>
[3] Tutorials/PipelineTutorials/Stackix.ipynb <br>
[4] Tutorials/DeepDives/ConfigTutorial.ipynb <br>

#### 🖥️ In Code

All the above steps run with one command in our pipeline:

In [5]:
# This does not return anything, the step updates internal attributes:
# It sets the private _datasets attribute
# It populates the results.datasets attribute with the preprocessed and transformed PyTorch Datasets (1 for each split)
vanillix.preprocess()

anno key: paired


### 2.2) The Fit Step
The fit step trains the autoencoder architecture and caputers an extensive set of training dynamics.  
The training and models itself are highly customizibale (n_layers, latent_dim, epochs, learning_rate, etc), refer to [section 5](#5-customize) or Tutorial [4] for even more details.  
During training, we caputure losses, latentspaces and reconstructions for each checkpoint epoch and store them in our result object.

---
[4] Tutorials/DeepDives/ConfigTutorial.ipynb <br>


In [6]:
# job of old make model
# calls self.Trainer class to init and train model
# populates self._model attribute with trained model
# populates self.result attribute with training results (model, losses, etc)
vanillix.fit()


Epoch 1 - Train Loss: 29.3901
Sub-losses: recon_loss: 29.3901
Epoch 1 - Valid Loss: 18.7674
Sub-losses: recon_loss: 18.7674
Epoch 2 - Train Loss: 26.3106
Sub-losses: recon_loss: 26.3106
Epoch 2 - Valid Loss: 19.3058
Sub-losses: recon_loss: 19.3058
Epoch 3 - Train Loss: 24.9490
Sub-losses: recon_loss: 24.9490
Epoch 3 - Valid Loss: 19.2850
Sub-losses: recon_loss: 19.2850
Epoch 4 - Train Loss: 23.8011
Sub-losses: recon_loss: 23.8011
Epoch 4 - Valid Loss: 18.8816
Sub-losses: recon_loss: 18.8816
Epoch 5 - Train Loss: 22.3824
Sub-losses: recon_loss: 22.3824
Epoch 5 - Valid Loss: 18.5436
Sub-losses: recon_loss: 18.5436
Epoch 6 - Train Loss: 21.4374
Sub-losses: recon_loss: 21.4374
Epoch 6 - Valid Loss: 18.0132
Sub-losses: recon_loss: 18.0132
Epoch 7 - Train Loss: 20.8209
Sub-losses: recon_loss: 20.8209
Epoch 7 - Valid Loss: 17.5613
Sub-losses: recon_loss: 17.5613
Epoch 8 - Train Loss: 20.2856
Sub-losses: recon_loss: 20.2856
Epoch 8 - Valid Loss: 17.1907
Sub-losses: recon_loss: 17.1907
Epoch 9 

### 2.3) The Predict Step
In this step we use the trained model to run a forward pass on our test dataset.  
This step can also take a `data` argument, so you can run predict on held-out data than the test split.  
If you provide new data, we will run preprocessing on this data, depending on the data format, see [1] for details.

[1] Tutorials/DeepDives/InputDataTutorials.ipynb

In [7]:
# job of old make predict
# if no data is passed, used the test split from preprocessing
# otherwise, uses the data passed, and preprocesses it
# updates self.result attribute with predictions (latent space, reconstructions, etc)
# returns a result object
result = vanillix.predict()

### 2.4) The Visualize Step
This steps takes the training results and creates visualizations.  
For example plots of the loss curve, or the latenspaces.
The plots are stored internally and can be shown with the method `show_plots`.  
See [Section Visualize](#4-visualize) for more details on working with created plots.

In [8]:
# job of old make visualize
vanillix.visualize()

### 2.5) The Run Step
This step combines all the above steps and returns a result object.  
Furthermore you cann pass a data argument which then will be used in the predict step instead of the test split.

In [9]:
# calls all the steps in one wrapper:
# vanillix.preprocess()
# vanillix.fit()
# vanillix.predict()
# vanillix.visualize
vanillix = acx.Vanillix(config=myconfig, data=my_datapackage)
result = vanillix.run()
# also possilbe to pass data: vanillix.run(data=<your-data>)

in handle_direct_user_data with data: <class 'autoencodix.data.datapackage.DataPackage'>
anno key: paired
Epoch 1 - Train Loss: 29.0977
Sub-losses: recon_loss: 29.0977
Epoch 1 - Valid Loss: 18.2546
Sub-losses: recon_loss: 18.2546
Epoch 2 - Train Loss: 26.4003
Sub-losses: recon_loss: 26.4003
Epoch 2 - Valid Loss: 18.5756
Sub-losses: recon_loss: 18.5756
Epoch 3 - Train Loss: 23.8130
Sub-losses: recon_loss: 23.8130
Epoch 3 - Valid Loss: 18.3248
Sub-losses: recon_loss: 18.3248
Epoch 4 - Train Loss: 23.1572
Sub-losses: recon_loss: 23.1572
Epoch 4 - Valid Loss: 18.1190
Sub-losses: recon_loss: 18.1190
Epoch 5 - Train Loss: 22.1004
Sub-losses: recon_loss: 22.1004
Epoch 5 - Valid Loss: 17.7050
Sub-losses: recon_loss: 17.7050
Epoch 6 - Train Loss: 21.6725
Sub-losses: recon_loss: 21.6725
Epoch 6 - Valid Loss: 17.4755
Sub-losses: recon_loss: 17.4755
Epoch 7 - Train Loss: 21.2129
Sub-losses: recon_loss: 21.2129
Epoch 7 - Valid Loss: 17.2909
Sub-losses: recon_loss: 17.2909
Epoch 8 - Train Loss: 20.3

### 2.6) Extra: The Evaluate Step
This step is not included, when you call `.run()`. Here we use the trained `latent space` and compare it to other dimension reduction techiniques like PCA, UMAP or a random baseline.  
In the very basic case you can specify a column in your annotation data which you want to use for a machine learning task e.g. train a classifier on a disease state or cancer type, etc. We then use the represenations (latentspace, UMAP, etc.) to train this classifier/regressor. You have a lot of options to customize this steps for your needs, like passing a sklearn model and defining evaluation metrics, please refer to [5] for a complete guide.  

[5] Tutorials/DeepDives/EvaluateTutorial.ipynb

In [10]:
# in our Mock DataFrame we have a column called "condition" 
# that refers to some clincial condition
vanillix.evaluate(params=["condition"]) #TODO should be also possibl ein config with annotation_columns

# next we can visualize the eval output with:
# TODO build method, to not expose private _visualizer
vanillix._visualizer.show_evaluation(
	param = "condition",
    metric="roc_auc_ovo"
)

Perform ML task with feature df: Latent
Latent
Perform ML task for target parameter: condition


KeyError: "None of [Index(['sample_0', 'sample_10', 'sample_102', 'sample_103', 'sample_104',\n       'sample_106', 'sample_107', 'sample_108', 'sample_11', 'sample_110',\n       ...\n       'sample_89', 'sample_9', 'sample_91', 'sample_92', 'sample_94',\n       'sample_95', 'sample_96', 'sample_97', 'sample_98', 'sample_99'],\n      dtype='object', length=350)] are in the [index]"

## 3) Inspect Results
Each step in the pipeline writes its results in the result object of the Vanillix instance.
In this section we explore how to access and make sense of the results.  
The attributes of the Result object are instances of a TrainingDynamics class.  
This class provied a standardized interface of how to access result from different split and epochs.


##### TrainingDynamics Object in Result
The `TrainingDynamics` object has the following form:  
`<epoch><split><data>`  

So, if you want to access the train loss for the 5th epoch, you would use:  
```python
result.loss.get(epoch=5, split="train")
```

##### Special Handling

* If an invalid split is provided, a `KeyError` is raised
* Negative epoch indices work like Python list indexing (-1 is the last epoch)
* If an epoch doesn't exist, an empty array or dictionary is returned

---
For more details on how to work with TrainingDynamics and the Result object, refer to [5]
[5]Tutorials/DeepDives/PipelineOutputTutorial.ipynb 



In [ ]:
# Get the latentspaces of the test split
rec_test = result.reconstructions.get(split="test")
# get sample_ids for this latent space
sample_ids = result.sample_ids.get(split="test")
# a utility wrapper to get the latend space as a dataframe
result.get_latent_df(epoch=-1, split="test")

## 4) Visualize Results
During the pipeline, we've already created all plots. To keep the notebook clean, we did not show the plots yet.  
For this you can call the `show_results` method to display the actual plots.  
Refer to [5] for more details on how to work with plots.

[5]Tutorials/DeepDives/PipelineOutputTutorial.ipynb 

In [ ]:
vanillix.show_result()

## 5) Customize

To customize the behavior of our pipeline, you adjust the configuration.  
There are two ways to work with the config:

1. Create a customized instance of the config class.  
2. Provide a `yaml` file and use the config class to read it.  

We will focus on option 1 and show a few examples. For a deeper dive into configurations, please refer to [4].  

In this section, we demonstrate how to change the following:  
- Scaling method  
- Number of layers in the autoencoder  
- Number of epochs  
- Learning rate  
- Latent dimensions  
- Retrieve information about all config parameters  

[4] Tutorials/DeepDives/ConfigTutorial.ipynb <br>


Each pipeline (Vanillix, Varix, Stackix, …) has a corresponding configuration class.  
All configuration classes inherit from a common `DefaultConfig` class, which defines all customizable parameters.  
Child classes override only those parameters that have proven to be sensible defaults for their specific pipeline.


In [ ]:
from autoencodix.configs import VanillixConfig
# we simply import the specific config an init it with custom parameters
custom_config = VanillixConfig(
    scaling="MINMAX", n_layers=4, epochs=23, learning_rate=1e4, latent_dim=4, data_case="Multi Bulk"
)
# to see a list of all parameters and defaults use:
custom_config.print_schema()

In [ ]:
# next we pass the config to our pipeline object
vanillix2 = acx.Vanillix(data=my_datapackage, config=custom_config)


## 6) Re-use, Save, Load

Lastly, we show how to re-use a trained pipeline, as well as how to save and load it.  
You can do the following:

- Use the trained pipeline to run the `predict` step with new data.  
- Obtain the latent space, apply transformations, and run the `decode` step of the trained pipeline.  
- Access the underlying PyTorch model and re-use it in any way you like.  
- Store the pipeline on disk or in the cloud, then reload it and continue using it.  


#### Run with New Data

In [ ]:
# for demonstration purposes we use the datapackage from step 1 again
# in real world scenarios you would use different data
result_new_data = vanillix.predict(data=my_datapackage)

#### Decode with latent space shift

In [ ]:
import numpy as np
import torch
latent = result.latentspaces.get(split="test", epoch=-1)
latent_shift = np.random.normal(0, 0.1, latent.shape)
new_latent = (latent + latent_shift).astype(np.float32)
vanillix.decode(torch.from_numpy(new_latent))

#### Obtain PyTorch Model

In [ ]:
model = result.model
# then do something with the model
print(model)

#### Save and Load

In [ ]:
import os
import glob

outpath = os.path.join("tutorial_res", "van.pkl")
vanillix.save(file_path=outpath, save_all=True)

folder = os.path.dirname(outpath)
pkl_files = glob.glob(os.path.join(folder, "*.pkl"))
model_files = glob.glob(os.path.join(folder, "*.pth"))

print("PKL files:", pkl_files)
print("Model files:", model_files)

# the load functionality automatically will build the pipeline object out of the three saved files
vanillix_loaded = acx.Vanillix.load(outpath)

In [ ]:
import os
import glob

outpath = os.path.join("tutorial_res", "van_all.pkl")
vanillix.save(file_path=outpath, save_all=False)

folder = os.path.dirname(outpath)
pkl_files = glob.glob(os.path.join(folder, "*.pkl"))
model_files = glob.glob(os.path.join(folder, "*.pth"))

print("PKL files:", pkl_files)
print("Model files:", model_files)

# the load functionality automatically will build the pipeline object out of the three saved files
vanillix_loaded = acx.Vanillix.load(outpath)

In [ ]:
vanillix_loaded.visualize()

In [ ]:
vanillix_loaded.predict(data=my_datapackage)
vanillix_loaded.show_result(split="test")

In [ ]:
vanillix_loaded.evaluate()

In [ ]:
# Downgrade matplotlib to a compatible version
%pip install matplotlib==3.4.3

# After running the above cell, restart the kernel and re-run the cell below.
ontix = acx.Ontix(ontologies=(1,2)).load(os.path.join("tutorial_res", "large-ontix.pkl"))

In [ ]:
n_epochs = 100
checkpoint_interval = 100
epoch = 99

(epoch + 1) % checkpoint_interval == 0 or epoch == n_epochs - 1

